In [ ]:
import sagemaker
from sagemaker import get_execution_role

print(sagemaker.__version__)
 
role = get_execution_role()
session = sagemaker.Session()

In [ ]:
bucket = session.default_bucket()

prefix = 'pascalvoc'

s3_train_data = 's3://{}/{}/input/train'.format(bucket, prefix)
s3_validation_data = 's3://{}/{}/input/validation'.format(bucket, prefix)
s3_output_location = 's3://{}/{}/output'.format(bucket, prefix)

print(s3_train_data)
print(s3_validation_data)

In [ ]:
import boto3
from sagemaker import image_uris

region = boto3.Session().region_name    
container = image_uris.retrieve('object-detection', region)
print(container)

In [ ]:
role = get_execution_role()

od = sagemaker.estimator.Estimator(container,
                                   role,
                                   instance_count=1,
                                   instance_type='ml.p3.2xlarge',
                                   output_path=s3_output_location,
                                   volume_size=8)

In [ ]:
od.set_hyperparameters(base_network='resnet-50',
                       use_pretrained_model=1,
                       num_classes=20,
                       epochs=30,
                       num_training_samples=16551,
                       mini_batch_size=90)

In [ ]:
train_data = sagemaker.TrainingInput(s3_train_data, distribution='FullyReplicated', 
                        content_type='application/x-recordio', s3_data_type='S3Prefix',
                        input_mode='Pipe')

validation_data = sagemaker.TrainingInput(s3_validation_data, distribution='FullyReplicated', 
                             content_type='application/x-recordio', s3_data_type='S3Prefix',
                             input_mode='Pipe')

data_channels = {'train': train_data, 'validation': validation_data}

In [ ]:
od.fit(inputs=data_channels)

In [ ]:
od_predictor = od.deploy(initial_instance_count = 1, instance_type = 'ml.c5.2xlarge')

In [ ]:
!wget -O test.jpg https://upload.wikimedia.org/wikipedia/commons/6/67/Chin_Village.jpg
with open('test.jpg', 'rb') as image:
    f = image.read()
    b = bytearray(f)

In [ ]:
import json

od_predictor.content_type = 'image/jpeg'
results = od_predictor.predict(b)
response = json.loads(results)

In [ ]:
print(response)

In [ ]:
def visualize_detection(img_file, dets, classes=[], thresh=0.6):
        """
        visualize detections in one image
        Parameters:
        ----------
        img : numpy.array
            image, in bgr format
        dets : numpy.array
            ssd detections, numpy.array([[id, score, x1, y1, x2, y2]...])
            each row is one object
        classes : tuple or list of str
            class names
        thresh : float
            score threshold
        """
        import random
        import matplotlib.pyplot as plt
        import matplotlib.image as mpimg

        img=mpimg.imread(img_file)
        plt.imshow(img)
        height = img.shape[0]
        width = img.shape[1]
        colors = dict()
        for det in dets:
            (klass, score, x0, y0, x1, y1) = det
            if score < thresh:
                continue
            cls_id = int(klass)
            if cls_id not in colors:
                colors[cls_id] = (random.random(), random.random(), random.random())
            xmin = int(x0 * width)
            ymin = int(y0 * height)
            xmax = int(x1 * width)
            ymax = int(y1 * height)
            rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                                 ymax - ymin, fill=False,
                                 edgecolor=colors[cls_id],
                                 linewidth=3.5)
            plt.gca().add_patch(rect)
            class_name = str(cls_id)
            if classes and len(classes) > cls_id:
                class_name = classes[cls_id]
            plt.gca().text(xmin, ymin - 2,
                            '{:s} {:.3f}'.format(class_name, score),
                            bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                    fontsize=12, color='white')
        plt.show()

In [ ]:
%matplotlib inline 

object_categories = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 
                     'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike', 'person', 
                     'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']

# Setting a threshold 0.20 will only plot detection results that have a confidence score greater than 0.20.
threshold = 0.30

# Visualize the detections.
visualize_detection('test.jpg', response['prediction'], object_categories, threshold)

In [ ]:
od_predictor.delete_endpoint()